In [ ]:
import ee

ee.Authenticate(force=True, scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/cloud-platform'])

# Initialize the library
ee.Initialize()
import numpy as np
from PIL import Image




Successfully saved authorization token.


In [6]:
# ==========================================
# 1. LOAD DHS CLUSTERS
# ==========================================

# Define paths nad names
ASSET_ID = 'projects/integrated-hawk-485001-k3/assets/ph_dhs_sample_50'
FOLDER_NAME = 'Sample_VIIRS_Chips'

# Load sample clusters
clusters = ee.FeatureCollection(ASSET_ID)

In [7]:
# ==========================================
# 2. DEFINE VIIRS PARAMETERS
# ==========================================
# We select 'average' radiance and create a single image from the collection
viirs_img = ee.ImageCollection("NOAA/VIIRS/DNB/ANNUAL_V22") \
    .filterDate('2022-01-01', '2022-12-31') \
    .select('average') \
    .median()

In [8]:
# ==========================================
# 3. PROCESS AND EXPORT
# ==========================================
# We convert the collection to a list to iterate in Python
cluster_list = clusters.getInfo()['features']
print(f"Submitting {len(cluster_list)} export tasks to Google Drive...")

for feature in cluster_list:
    props = feature['properties']
    cluster_id = props.get('DHSCLUST')
    
    # Logic for buffer size based on your project requirements
    # 'U' for Urban (2km), 'R' for Rural (5km)
    is_urban = props.get('URBAN_RURA') == 'U'
    radius = 2000 if is_urban else 5000
    
    # Create the bounding box for the chip
    lon, lat = feature['geometry']['coordinates']
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(radius).bounds()
    
    # Set up the export task
    task_name = f'VIIRS_Cluster_{cluster_id}'
    
    task = ee.batch.Export.image.toDrive(
        image=viirs_img,
        description=task_name,
        folder=FOLDER_NAME,
        region=roi,
        scale=463, # Native VIIRS resolution
        fileFormat='GeoTIFF',
        crs='EPSG:4326',
        maxPixels=1e9
    )
    
    task.start()
    print(f"Started: {task_name} (Radius: {radius}m)")

print("\nAll tasks submitted. You can monitor progress at: https://code.earthengine.google.com/tasks")

Submitting 50 export tasks to Google Drive...
Started: VIIRS_Cluster_441 (Radius: 2000m)
Started: VIIRS_Cluster_450 (Radius: 2000m)
Started: VIIRS_Cluster_442 (Radius: 5000m)
Started: VIIRS_Cluster_491 (Radius: 5000m)
Started: VIIRS_Cluster_512 (Radius: 5000m)
Started: VIIRS_Cluster_558 (Radius: 2000m)
Started: VIIRS_Cluster_563 (Radius: 2000m)
Started: VIIRS_Cluster_600 (Radius: 5000m)
Started: VIIRS_Cluster_532 (Radius: 5000m)
Started: VIIRS_Cluster_651 (Radius: 2000m)
Started: VIIRS_Cluster_638 (Radius: 5000m)
Started: VIIRS_Cluster_667 (Radius: 5000m)
Started: VIIRS_Cluster_699 (Radius: 5000m)
Started: VIIRS_Cluster_727 (Radius: 5000m)
Started: VIIRS_Cluster_705 (Radius: 5000m)
Started: VIIRS_Cluster_843 (Radius: 2000m)
Started: VIIRS_Cluster_784 (Radius: 2000m)
Started: VIIRS_Cluster_832 (Radius: 5000m)
Started: VIIRS_Cluster_876 (Radius: 2000m)
Started: VIIRS_Cluster_895 (Radius: 5000m)
Started: VIIRS_Cluster_909 (Radius: 5000m)
Started: VIIRS_Cluster_948 (Radius: 2000m)
Started: